# GlobusWorld 2018 JupyterHub Tutorial

This tutorial will show you how to pull Globus Auth tokens from the Jupyter Notebook Server environment and use them to interact with different REST APIs secured with Globus Auth. The code in this notebook are very pedantic for clarity at each step; much of this code could be encapsulated in Python packages to simplify the work in the notebook. Research project could also add more semantically meaningful layers for their users (e.g., `MyData.download()` rather 
than `request.get('https://example.org/mydata.csv`)`.

## Import Packages

In [2]:
# These are to get the tokens
import os
import pickle
import base64

# Much of what we're dealing with is JSON
import json

# We're going to be making explicit HTTPS calls
import requests

# This is to work with data for our example
import numpy
import matplotlib

## Get Tokens

The tokens are passed into the environment `base64` encoded pickled Python dictionary assigned to the `GLOBUS_DATA` variable. We'll grab the variable and unpack it. 

In [3]:
globus_env_data = os.getenv('GLOBUS_DATA')

In [5]:
# This is the raw base64 encoded data
print(globus_env_data)

gAN9cQAoWAkAAABjbGllbnRfaWRxAVgkAAAANGI3MjUyNDUtOTU3Ni00ODhhLThkY2ItNDdkZDY4MWFlZDE4cQJYBgAAAHRva2Vuc3EDfXEEKFgPAAAAYXV0aC5nbG9idXMub3JncQV9cQYoWAUAAABzY29wZXEHWBQAAABwcm9maWxlIG9wZW5pZCBlbWFpbHEIWAwAAABhY2Nlc3NfdG9rZW5xCVhkAAAAQWd4UEpPajZHejdwcFlYTXg0ejF5YlhEOWJFTUdlMWU0UE9rbG9vQkJ2akdtV1A0ZzJIOENONjhuYktrbTNhb1FHT3ZibVlFcGpRVndYdHFZQk83elRLYTIwQ1hndjNmZ3Y3anEKWA0AAAByZWZyZXNoX3Rva2VucQtOWAoAAAB0b2tlbl90eXBlcQxYBgAAAEJlYXJlcnENWBIAAABleHBpcmVzX2F0X3NlY29uZHNxDkoSneBadVgTAAAAcGV0cmVsX2h0dHBzX3NlcnZlcnEPfXEQKGgHWEcAAABodHRwczovL2F1dGguZ2xvYnVzLm9yZy9zY29wZXMvNTZjZWFjMjktZTk4YS00NDBhLWE1OTQtYjQxZTdhMDg0YjYyL2FsbHERaAlYWgAAAEFnM2o2N1ZNN21OelFORWR3M0J4OU9NandwSzUyNldreDdqcE5LNHJlZ2VsYU84S3I5VFZDSk9yRXpkd3lLTzlKMTFZcmo0VjRrN3JYalNYZGdKUDJUZDI3eHESaAtOaAxYBgAAAEJlYXJlcnETaA5KEp3gWnVYFwAAAHRyYW5zZmVyLmFwaS5nbG9idXMub3JncRR9cRUoaAdYMQAAAHVybjpnbG9idXM6YXV0aDpzY29wZTp0cmFuc2Zlci5hcGkuZ2xvYnVzLm9yZzphbGxxFmgJWFoAAABBZ3EwMnhkejlQbjNLN05OdzhrcEs0bjB6ajJtMFd6RGtNeHB6NjFhbW9PS2xLN09CbUlxQ083RWRNM3A2

In [6]:
# Now we have the pickled tokens
pickled_tokens = base64.b64decode(globus_env_data)

In [8]:
# Unpickle and get the dictionary
tokens = pickle.loads(pickled_tokens)
isinstance(tokens, dict)

True

## Look Inside the Tokens

Depending on the JupyterHub configuration, there will be different numbers of tokens. For this tutorial we have our identity token as __[JSON Web Token (JWT)](https://tools.ietf.org/html/rfc7519)__, and then access tokens for different Resource Servers and scopes, like getting our profile from Globus Auth.

In [9]:
print(json.dumps(tokens, indent=4, sort_keys=True))

{
    "client_id": "4b725245-9576-488a-8dcb-47dd681aed18",
    "id_token": {
        "at_hash": "aimR0boKV2D-USfcPskq0zVXq1Eyux8C4c3IhrsESjY",
        "aud": "4b725245-9576-488a-8dcb-47dd681aed18",
        "email": "rick@globus.org",
        "exp": 1524669714,
        "iat": 1524496914,
        "identity_provider": "41143743-f3c8-4d60-bbdb-eeecaba85bd9",
        "identity_provider_display_name": "Globus ID",
        "iss": "https://auth.globus.org",
        "name": "Rick Wagner",
        "organization": "Globus",
        "preferred_username": "rpwagner@globusid.org",
        "sub": "c4e16e62-d274-11e5-b9e9-cf0225068ce8"
    },
    "tokens": {
        "auth.globus.org": {
            "access_token": "AgxPJOj6Gz7ppYXMx4z1ybXD9bEMGe1e4POklooBBvjGmWP4g2H8CN68nbKkm3aoQGOvbmYEpjQVwXtqYBO7zTKa20CXgv3fgv7j",
            "expires_at_seconds": 1524669714,
            "refresh_token": null,
            "scope": "profile openid email",
            "token_type": "Bearer"
        },
        "petrel_

## Using the Tokens

Now we can get started talking to different servers. For the purpose of this tutorial we're going to show how the tokens can be passed as HTTP headers. Much of this can also be done with the __[Globus Python SDK] (http://globus-sdk-python.readthedocs.io/en/stable/)__.

### User Info

First, let's use our `auth.globus.org` token to get our __[OAuth2 user information](https://docs.globus.org/api/auth/reference/#get_or_post_v2_oauth2_userinfo_resource)__. We assemble the header with the appropriate access token and do an HTTP `GET` on the resource. (N.B. We're using HTTPS even though we refer to the HTTP methods.) 

In [11]:
# Create the header
headers = {'Authorization':'Bearer '+ tokens['tokens']['auth.globus.org']['access_token']}

# Get the user info as JSON
user_info = requests.get('https://auth.globus.org/v2/oauth2/userinfo',headers=headers).json()

# Look at the response
print(json.dumps(user_info, indent=4, sort_keys=True))

{
    "email": "rick@globus.org",
    "identity_provider": "41143743-f3c8-4d60-bbdb-eeecaba85bd9",
    "identity_provider_display_name": "Globus ID",
    "name": "Rick Wagner",
    "organization": "Globus",
    "preferred_username": "rpwagner@globusid.org",
    "sub": "c4e16e62-d274-11e5-b9e9-cf0225068ce8"
}


### Identities

This is the __[Globus Auth API resource for identities](https://docs.globus.org/api/auth/reference/#v2_api_identities_resources)__. We use a similar process, but we're doing a `GET` on a specific identity, our own.

In [12]:
identity = requests.get('https://auth.globus.org/v2/api/identities/' + user_info['sub'],headers=headers).json()
print(json.dumps(identity, indent=4, sort_keys=True))

{
    "identity": {
        "email": "rick@globus.org",
        "id": "c4e16e62-d274-11e5-b9e9-cf0225068ce8",
        "identity_provider": "41143743-f3c8-4d60-bbdb-eeecaba85bd9",
        "name": "Rick Wagner",
        "organization": "Globus",
        "status": "used",
        "username": "rpwagner@globusid.org"
    }
}


## A Minimal Workflow

Let's replicate a basic workflow:

1. Pull down some data
1. Plot it
1. Save the plot to a webserver
1. Share the link

### Pulling Down a CSV File

### Plot the Data

### Save the Plot

### Check the Link

In [12]:
# http://statweb.stanford.edu/~sabatti/data.html
net_data = requests.get('http://hci.stanford.edu/courses/cs448b/data/ipasn/cs448b_ipasn.csv').text